# NLP in Pyspark's MLlib Project

## Fake Job Posting Predictions

Indeed.com has just hired you to create a system that automatically flags suspicious job postings on it's website. It has recently seen an influx of fake job postings that is negativley impacting it's customer experience. Becuase of the high volume of job postings it receives everyday, their employees do have the capacity to check every posting so they would like prioritize which postings to review before deleting it. 

#### Your task
Use the attached dataset with NLP to create an alogorthim which automatically flags suspicious posts for review. 

#### The data
This dataset contains 18K job descriptions out of which about 800 are fake. The data consists of both textual information and meta-information about the jobs.

**Data Source:** https://www.kaggle.com/shivamb/real-or-fake-fake-jobposting-prediction

#### Have fun!

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Flag Suspicious Posts').getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print('You are working with', cores, 'core(s)')
spark

You are working with 1 core(s)


In [2]:
# !pip install sparknlp
# import sparknlp
# from sparknlp.annotator import *
# from sparknlp.pretrained import *

# spark = sparknlp.start()

In [3]:
from pyspark.ml.feature import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.classification import *
from pyspark.ml.evaluation import *
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline 

In [4]:
fake_jobs = spark.read.csv('Datasets/fake_job_postings.csv', inferSchema=True, header=True)

In [5]:
fake_jobs.limit(5).toPandas()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,None,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,None,0,1,0,Other,Internship,None,None,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,None,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,None,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",None,None,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,None,0,1,0,None,None,None,None,None,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,None,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",None,None,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [6]:
fake_jobs.printSchema()

root
 |-- job_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- location: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary_range: string (nullable = true)
 |-- company_profile: string (nullable = true)
 |-- description: string (nullable = true)
 |-- requirements: string (nullable = true)
 |-- benefits: string (nullable = true)
 |-- telecommuting: string (nullable = true)
 |-- has_company_logo: string (nullable = true)
 |-- has_questions: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- required_experience: string (nullable = true)
 |-- required_education: string (nullable = true)
 |-- industry: string (nullable = true)
 |-- function: string (nullable = true)
 |-- fraudulent: string (nullable = true)



In [7]:
def null_value_calc(df):
    null_columns_counts = []
    numRows = df.count()
    for k in df.columns:
        nullRows = df.where(col(k).isNull()).count()
        if(nullRows > 0):
            temp = k, nullRows,(nullRows/numRows)*100
            null_columns_counts.append(temp)
    return(null_columns_counts)

null_columns_calc_list = sorted(null_value_calc(fake_jobs), reverse=True, key=lambda x: x[-1])

spark.createDataFrame(null_columns_calc_list, ['Column_Name', 'Null_Values_Count','Null_Value_Percent']). \
withColumn('Null_Value_Percent', round('Null_Value_Percent', 3)).show()

+-------------------+-----------------+------------------+
|        Column_Name|Null_Values_Count|Null_Value_Percent|
+-------------------+-----------------+------------------+
|       salary_range|            15011|            83.954|
|         department|            11547|            64.581|
| required_education|             7748|            43.333|
|           benefits|             6966|             38.96|
|required_experience|             6723|            37.601|
|           function|             6317|             35.33|
|           industry|             4831|            27.019|
|    company_profile|             3308|            18.501|
|    employment_type|             3292|            18.412|
|       requirements|             2573|             14.39|
|           location|              346|             1.935|
|         fraudulent|              176|             0.984|
|      telecommuting|               89|             0.498|
|      has_questions|               30|             0.16

In [8]:
fake_jobs.groupBy('fraudulent').count().orderBy('count', ascending=False).show()

+--------------------+-----+
|          fraudulent|count|
+--------------------+-----+
|                   0|16080|
|                   1|  886|
|                null|  176|
|           Full-time|   73|
|Hospital & Health...|   55|
|   Bachelor's Degree|   53|
|         Engineering|   26|
| perform quality ...|   17|
|         Unspecified|   15|
|    Mid-Senior level|   15|
|           Associate|   14|
|               Sales|   14|
|Information Techn...|   13|
|           Marketing|   13|
| passionate about...|   13|
|            Internet|   12|
|   Computer Software|   12|
|      Not Applicable|   11|
|We offer an excel...|   11|
| además con el fi...|   10|
+--------------------+-----+
only showing top 20 rows



In [9]:
fake_jobs = fake_jobs.filter('fraudulent IN(0, 1)')

In [10]:
fake_jobs.groupBy('fraudulent').count().orderBy('count', ascending=False).show()

+----------+-----+
|fraudulent|count|
+----------+-----+
|         0|16080|
|         1|  886|
+----------+-----+



In [11]:
fake_jobs = fake_jobs.drop('job_id', 'salary_range')

In [12]:
len(fake_jobs.columns)

16

In [13]:
orig_len = fake_jobs.count()
drop_len = fake_jobs.dropna(thresh=10).count()
print('Total Rows that contain 10 null values:', orig_len-drop_len)
print('Percentage of Rows that contain 10 null values value:', (orig_len-drop_len)/orig_len)

Total Rows that contain 10 null values: 2187
Percentage of Rows that contain 10 null values value: 0.12890486856065073


In [14]:
fake_jobs = fake_jobs.dropna(thresh=10)
fake_jobs = fake_jobs.na.fill(' ')

In [15]:
fake_jobs.select([count(when(isnan(c), c)).alias(c) for c in fake_jobs.columns]).limit(5).toPandas()

,title,location,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

fake_jobs = fake_jobs.select(concat_ws(' ', fake_jobs.title, fake_jobs.location, fake_jobs.department, fake_jobs.company_profile, 
                          fake_jobs.description, fake_jobs.requirements, fake_jobs.benefits, fake_jobs.employment_type,
                          fake_jobs.required_experience, fake_jobs.required_education, fake_jobs.industry, fake_jobs.function). \
                alias('text'), 'fraudulent')

In [17]:
fake_jobs.toPandas().head(3)

text  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   Marketing Intern US, NY, New York Marketing We're Food52, and we've created a groundbreaking and award-winning cooking site. We support, connect, and celebrate home cooks, and give them everything they need in one place.We have a top editorial, business, and engineering team. We're focused on using technology to find new and better ways to connect people around their specific food interests, and to offer them superb, highly curated information about food and cooking. We attract t

In [18]:
fake_jobs = fake_jobs.withColumn('text', regexp_replace(col('text'), '[^A-Za-z]+', ' '))

fake_jobs = fake_jobs.withColumn('text', lower(regexp_replace(col('text'), '\s+', ' ')))

In [19]:
regex_tokenizer = RegexTokenizer(inputCol='text', outputCol='words', pattern='\\W')

remover = StopWordsRemover(inputCol=regex_tokenizer.getOutputCol(), outputCol='filtered')

indexer = StringIndexer(inputCol='fraudulent', outputCol='label')


pipeline = Pipeline(stages=[regex_tokenizer, remover, indexer])
data_prep_pl = pipeline.fit(fake_jobs)

feature_data = data_prep_pl.transform(fake_jobs)

In [20]:
# import sparknlp
# from sparknlp.base import *
# from sparknlp.annotator import *
# from pyspark.ml import Pipeline 

# documentAssembler = DocumentAssembler().setInputCol('text').setOutputCol('document')

# sentenceDetector = SentenceDetector().setInputCols(['document']).setOutputCol('sentence')

# regexTokenizer = Tokenizer().setInputCols(['sentence']).setOutputCol('tokens')

# stop_words = StopWordsCleaner.pretrained('stopwords_iso','en').setInputCols(['tokens']).setOutputCol('cleanTokens')

# lemma = LemmatizerModel.pretrained('lemma_lines', 'en').setInputCols(['cleanTokens']).setOutputCol('lemma')



# pipeline = Pipeline(stages=[documentAssembler, sentenceDetector, regex_tokenizer, stop_words, lemma])

# feature_data = pipeline.fit(fake_jobs).transform(fake_jobs)

In [21]:
# pipeline = PretrainedPipeline('spellcheck_dl_pipeline', lang = 'en')

# def spell_check(x):
#     ' '.join(pipeline.annotate(text)[0]['checked'])

In [22]:
feature_data.toPandas().head(2)

text  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   marketing intern us ny new york marketing we re food and we ve created a groundbreaking and award winning cooking site we support connect and celebrate home cooks and give them everything they need in one place we have a top editorial business and engineering team we re focused on using technology to find new and better ways to connect people around their specific food interests and to offer them superb highly curated information about food and cooking we attract the most talented home cooks and contributors in the country we also publish well known professionals like mario batali gwyneth paltrow and danny meyer and we have partnerships with whole foods market and random house food has been named the best food website by the james beard foundation and iacp and has been featured in the new york

In [23]:
word2Vec = Word2Vec(vectorSize=5, minCount=0, inputCol='filtered', outputCol='features')

model = word2Vec.fit(feature_data)

W2VfeaturizedData = model.transform(feature_data)

scaler = MinMaxScaler(inputCol='features', outputCol='scaledFeatures')
scalerModel = scaler.fit(W2VfeaturizedData)

scaled_data = scalerModel.transform(W2VfeaturizedData)

W2VfeaturizedData = scaled_data.select('fraudulent','text','label','scaledFeatures')
W2VfeaturizedData = W2VfeaturizedData.withColumnRenamed('scaledFeatures','features')

W2VfeaturizedData.name = 'W2VfeaturizedData'

In [24]:
def ClassTrainEval(classifier, features, classes, train, test):

    def FindMtype(classifier):
        # Intstantiate Model
        M = classifier
        # Learn what it is
        Mtype = type(M).__name__
        
        return Mtype
    
    Mtype = FindMtype(classifier)
    

    def IntanceFitModel(Mtype, classifier, classes, features, train):
        
        if Mtype == 'OneVsRest':
            # instantiate the base classifier.
            lr = LogisticRegression()
            # instantiate the One Vs Rest Classifier.
            OVRclassifier = OneVsRest(classifier=lr)
#             fitModel = OVRclassifier.fit(train)
            # Add parameters of your choice here:
            paramGrid = ParamGridBuilder() \
                .addGrid(lr.regParam, [0.1, 0.01]) \
                .build()
            #Cross Validator requires the following parameters:
            crossval = CrossValidator(estimator=OVRclassifier,
                                      estimatorParamMaps=paramGrid,
                                      evaluator=BinaryClassificationEvaluator(),
                                      numFolds=5) # 3 is best practice
            # Run cross-validation, and choose the best set of parameters.
            fitModel = crossval.fit(train)
            return fitModel
        if Mtype == 'MultilayerPerceptronClassifier':
            # specify layers for the neural network:
            # input layer of size features, two intermediate of features+1 and same size as features
            # and output of size number of classes
            # Note: crossvalidator cannot be used here
            features_count = len(features[0][0])
            layers = [features_count, features_count+1, features_count, classes]
            MPC_classifier = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)
            fitModel = MPC_classifier.fit(train)
            return fitModel
        if Mtype in('LinearSVC','GBTClassifier') and classes != 2: # These classifiers currently only accept binary classification
            print(Mtype,' could not be used because PySpark currently only accepts binary classification data for this algorithm')
            return
        if Mtype in('LogisticRegression','NaiveBayes','RandomForestClassifier','GBTClassifier','LinearSVC','DecisionTreeClassifier'):
  
            # Add parameters of your choice here:
            if Mtype in('LogisticRegression'):
                paramGrid = (ParamGridBuilder() \
#                              .addGrid(classifier.regParam, [0.1, 0.01]) \
                             .addGrid(classifier.maxIter, [10, 15, 20])
                             .build())
                
            # Add parameters of your choice here:
            if Mtype in('NaiveBayes'):
                paramGrid = (ParamGridBuilder() \
                             .addGrid(classifier.smoothing, [0.0, 0.2, 0.4, 0.6]) \
                             .build())
                
            # Add parameters of your choice here:
            if Mtype in('RandomForestClassifier'):
                paramGrid = (ParamGridBuilder() \
                               .addGrid(classifier.maxDepth, [2, 5, 10])
#                                .addGrid(classifier.maxBins, [5, 10, 20])
#                                .addGrid(classifier.numTrees, [5, 20, 50])
                             .build())
                
            # Add parameters of your choice here:
            if Mtype in('GBTClassifier'):
                paramGrid = (ParamGridBuilder() \
#                              .addGrid(classifier.maxDepth, [2, 5, 10, 20, 30]) \
#                              .addGrid(classifier.maxBins, [10, 20, 40, 80, 100]) \
                             .addGrid(classifier.maxIter, [10, 15, 50, 100])
                             .build())
                
            # Add parameters of your choice here:
            if Mtype in('LinearSVC'):
                paramGrid = (ParamGridBuilder() \
                             .addGrid(classifier.maxIter, [10, 15]) \
                             .addGrid(classifier.regParam, [0.1, 0.01]) \
                             .build())
            
            # Add parameters of your choice here:
            if Mtype in('DecisionTreeClassifier'):
                paramGrid = (ParamGridBuilder() \
#                              .addGrid(classifier.maxDepth, [2, 5, 10, 20, 30]) \
                             .addGrid(classifier.maxBins, [10, 20, 40, 80, 100]) \
                             .build())
            
            #Cross Validator requires all of the following parameters:
            crossval = CrossValidator(estimator=classifier,
                                      estimatorParamMaps=paramGrid,
                                      evaluator=BinaryClassificationEvaluator(),
                                      numFolds=5) # 3 + is best practice
            # Fit Model: Run cross-validation, and choose the best set of parameters.
            fitModel = crossval.fit(train)
            return fitModel
    
    fitModel = IntanceFitModel(Mtype, classifier, classes, features, train)
    
    # Print feature selection metrics
    if fitModel is not None:
        
        if Mtype in('OneVsRest'):
            # Get Best Model
            BestModel = fitModel.bestModel
            print(' ')
            print('\033[1m' + Mtype + '\033[0m')
            # Extract list of binary models
            models = BestModel.models
            for model in models:
                print('\033[1m' + 'Intercept: '+ '\033[0m', model.intercept,'\033[1m' + '\nCoefficients:'+ '\033[0m', model.coefficients)

        if Mtype == 'MultilayerPerceptronClassifier':
            print('')
            print('\033[1m' + Mtype,' Weights'+ '\033[0m')
            print('\033[1m' + 'Model Weights: '+ '\033[0m', fitModel.weights.size)
            print('')

        if Mtype in('DecisionTreeClassifier', 'GBTClassifier','RandomForestClassifier'):
            # FEATURE IMPORTANCES
            # Estimate of the importance of each feature.
            # Each feature’s importance is the average of its importance across all trees 
            # in the ensemble The importance vector is normalized to sum to 1. 
            # Get Best Model
            BestModel = fitModel.bestModel
            print(' ')
            print('\033[1m' + Mtype,' Feature Importances'+ '\033[0m')
            print('(Scores add up to 1)')
            print('Lowest score is the least important')
            print(' ')
            print(BestModel.featureImportances)
            
            if Mtype in('DecisionTreeClassifier'):
                global DT_featureimportances
                DT_featureimportances = BestModel.featureImportances.toArray()
                global DT_BestModel
                DT_BestModel = BestModel
            if Mtype in('GBTClassifier'):
                global GBT_featureimportances
                GBT_featureimportances = BestModel.featureImportances.toArray()
                global GBT_BestModel
                GBT_BestModel = BestModel
            if Mtype in('RandomForestClassifier'):
                global RF_featureimportances
                RF_featureimportances = BestModel.featureImportances.toArray()
                global RF_BestModel
                RF_BestModel = BestModel

        if Mtype in('LogisticRegression'):
            # Get Best Model
            BestModel = fitModel.bestModel
            print(' ')
            print('\033[1m' + Mtype,' Coefficient Matrix'+ '\033[0m')
            print('You should compares these relative to eachother')
            print('Coefficients: \n' + str(BestModel.coefficientMatrix))
            print('Intercept: ' + str(BestModel.interceptVector))
            global LR_coefficients
            LR_coefficients = BestModel.coefficientMatrix.toArray()
            global LR_BestModel
            LR_BestModel = BestModel

        if Mtype in('LinearSVC'):
            # Get Best Model
            BestModel = fitModel.bestModel
            print(' ')
            print('\033[1m' + Mtype,' Coefficients'+ '\033[0m')
            print('You should compares these relative to eachother')
            print('Coefficients: \n' + str(BestModel.coefficients))
            global LSVC_coefficients
            LSVC_coefficients = BestModel.coefficients.toArray()
            global LSVC_BestModel
            LSVC_BestModel = BestModel
        
   
    # Set the column names to match the external results dataframe that we will join with later:
    columns = ['Classifier', 'Result']
    
    if Mtype in('LinearSVC','GBTClassifier') and classes != 2:
        Mtype = [Mtype] # make this a list
        score = ['N/A']
        result = spark.createDataFrame(zip(Mtype, score), schema=columns)
    else:
        predictions = fitModel.transform(test)
        MC_evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC') # redictionCol='prediction',
        accuracy = (MC_evaluator.evaluate(predictions))*100
        Mtype = [Mtype] # make this a string
        score = [str(accuracy)] #make this a string and convert to a list
        result = spark.createDataFrame(zip(Mtype, score), schema=columns)
        result = result.withColumn('Result', result.Result.substr(0, 5))
        
    return result
    #Also returns the fit model important scores or p values

In [25]:
classifiers = [
                LogisticRegression()
                ,OneVsRest()
               ,LinearSVC()
               ,NaiveBayes()
               ,RandomForestClassifier()
               ,GBTClassifier()
               ,DecisionTreeClassifier()
               ,MultilayerPerceptronClassifier()
              ]

In [ ]:
train = W2VfeaturizedData.sampleBy('label', fractions={0.0: 0.7, 1.0: 0.7}, seed=16647) 
test = W2VfeaturizedData.subtract(train)

features = W2VfeaturizedData.select(['features']).collect()
class_count = W2VfeaturizedData.select(countDistinct('label')).collect()
classes = class_count[0][0]

columns = ['Classifier', 'Result']
vals = [('Place Holder','N/A')]
results = spark.createDataFrame(vals, columns)

for classifier in classifiers:
    new_result = ClassTrainEval(classifier, features, classes, train, test)
    results = results.union(new_result)

results = results.where("Classifier!='Place Holder'")
print(results.show(truncate=False))

 
LogisticRegression  Coefficient Matrix
You should compares these relative to eachother
Coefficients: 
DenseMatrix([[-0.88329853, -8.12662828, -4.42253355,  3.47656663,  3.07098487]])

Intercept: [1.3010120391442799]
 
OneVsRest
Intercept:  1.279021407332071 
Coefficients: [-2.3430055667243765,5.110206005470308,1.075032130410985,-1.5474437444272449,-0.9419807272019671]
Intercept:  -1.2790214073320674 
Coefficients: [2.3430055667243765,-5.110206005470308,-1.0750321304109898,1.5474437444272442,0.9419807272019641]
 
LinearSVC  Coefficients
You should compares these relative to eachother
Coefficients: 
[-0.02248529327797267,-0.014324214244473138,-0.023552191780259486,0.013239036096754636,0.012966542816556303]
 
RandomForestClassifier  Feature Importances
(Scores add up to 1)
Lowest score is the least important
 
(5,[0,1,2,3,4],[0.22163170826389714,0.19145855614888813,0.1830363620487308,0.1619178896715437,0.24195548386694035])
 
GBTClassifier  Feature Importances
(Scores add up to 1)
Lowes

In [ ]:
predictions = RF_BestModel.transform(test)

In [ ]:
print('Predicted Failures:')
predictions.select('text', 'fraudulent', 'prediction').filter('prediction=0').toPandas().head(3)

In [ ]:
print('Predicted Success:')
predictions.select('text', 'fraudulent', 'prediction').filter('prediction=1').toPandas().head(3)